<a href="https://colab.research.google.com/github/Neel7317/Deep-Learning-/blob/master/Language_translation_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
import numpy as np


In [3]:
batch_size=64#batch size for traning
epochs=100# No of Epochs
latent_dim=256  #Diamension for encoding 
num_sample=10000
data_path='fra.txt'

In [4]:
#vecorized the data
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path,'r',encoding='utf-8') as f:
  lines=f.read().split('\n')
for line in lines[:min(num_sample,len(lines)-1)]:
  input_text, target_text, _ = line.split('\t')
  target_text='\t'+ target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)


In [11]:
len(input_characters)

71

In [12]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)#71
num_decoder_tokens=len(target_characters)#93
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [13]:
print('Number of samples:',len(input_texts))
print('Number of unique input tokens:',num_encoder_tokens)
print('Number of unique output tokens:',num_decoder_tokens)
print('Max sequence length for inputs:',max_encoder_seq_length)
print('max sequence length for outputs:',max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
max sequence length for outputs: 59


In [14]:
input_token_index=dict([(char,i)for i,char in enumerate(input_characters)])
target_token_index=dict([(char,i) for i,char in enumerate(target_characters)])

In [15]:
input_token_index,target_token_index

({' ': 0,
  '!': 1,
  '"': 2,
  '$': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  ',': 7,
  '-': 8,
  '.': 9,
  '0': 10,
  '1': 11,
  '2': 12,
  '3': 13,
  '5': 14,
  '7': 15,
  '8': 16,
  '9': 17,
  ':': 18,
  '?': 19,
  'A': 20,
  'B': 21,
  'C': 22,
  'D': 23,
  'E': 24,
  'F': 25,
  'G': 26,
  'H': 27,
  'I': 28,
  'J': 29,
  'K': 30,
  'L': 31,
  'M': 32,
  'N': 33,
  'O': 34,
  'P': 35,
  'Q': 36,
  'R': 37,
  'S': 38,
  'T': 39,
  'U': 40,
  'V': 41,
  'W': 42,
  'Y': 43,
  'a': 44,
  'b': 45,
  'c': 46,
  'd': 47,
  'e': 48,
  'f': 49,
  'g': 50,
  'h': 51,
  'i': 52,
  'j': 53,
  'k': 54,
  'l': 55,
  'm': 56,
  'n': 57,
  'o': 58,
  'p': 59,
  'q': 60,
  'r': 61,
  's': 62,
  't': 63,
  'u': 64,
  'v': 65,
  'w': 66,
  'x': 67,
  'y': 68,
  'z': 69,
  'é': 70},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '$': 4,
  '%': 5,
  '&': 6,
  "'": 7,
  '(': 8,
  ')': 9,
  ',': 10,
  '-': 11,
  '.': 12,
  '0': 13,
  '1': 14,
  '2': 15,
  '3': 16,
  '5': 17,
  '8': 18,
  '9': 19,
  ':': 20,
  '?'

In [17]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
decoder_target_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')

In [18]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  for t,char in enumerate(input_text):
    encoder_input_data[i,t,input_token_index[char]]=1.
  encoder_input_data[i,t+1:,input_token_index[' ']]=1.
  for t,char in enumerate(target_text):
    #decoder_target_data is ahead of decoder_input_data by one timestamp
    decoder_input_data[i,t,target_token_index[char]]=1.
    if t > 0:
      #decoder_target_data will be ahead by one time stamp 
      #and will not include the start charater
      decoder_target_data[i,t-1,target_token_index[char]]=1.
  decoder_input_data[i,t+1:,target_token_index[' ']]=1.
  decoder_target_data[i,t:,target_token_index[' ']]=1.


In [19]:
encoder_input_data[0].shape

(15, 71)

In [21]:
#define on input sequence and process it
encoder_inputs=Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)
#we discard 'encoder outputs ' and only keep the states
encoder_states=[state_h,state_c]

In [22]:
#setup the decoder ,using  encoder_state as initial state
decoder_inputs=Input(shape=(None,num_decoder_tokens))
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [23]:
#define the model that will turn 'encoder_input_data' & 'decoder_input_data ' into 'decoder_target_data'


In [24]:
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)

In [25]:
#run traning
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [26]:
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 11s 21ms/step - loss: 1.4971 - accuracy: 0.7036 - val_loss: 1.0217 - val_accuracy: 0.7218
Epoch 2/100
125/125 [==============================] - 2s 15ms/step - loss: 0.8658 - accuracy: 0.7664 - val_loss: 0.8152 - val_accuracy: 0.7727
Epoch 3/100
125/125 [==============================] - 2s 15ms/step - loss: 0.6813 - accuracy: 0.8090 - val_loss: 0.6954 - val_accuracy: 0.7964
Epoch 4/100
125/125 [==============================] - 2s 15ms/step - loss: 0.5883 - accuracy: 0.8302 - val_loss: 0.6401 - val_accuracy: 0.8106
Epoch 5/100
125/125 [==============================] - 2s 15ms/step - loss: 0.5322 - accuracy: 0.8435 - val_loss: 0.5928 - val_accuracy: 0.8270
Epoch 6/100
125/125 [==============================] - 2s 15ms/step - loss: 0.4996 - accuracy: 0.8528 - val_loss: 0.5589 - val_accuracy: 0.8375
Epoch 7/100
125/125 [==============================] - 2s 15ms/step - loss: 0.4723 - accuracy: 0.8608 - val_loss: 0.5414 - val_accuracy

In [27]:
#next inference mode(sampling)
#Here's the drill
#1) encode input and retrive initial decoder stste
#2) run one step of decoder with this initial state
#and s tatrt of sequence taken as target
#output will be the next target token
#3) Repeat with the current target tokenm and current states

In [30]:
#defining the sample models
encoder_model=Model(encoder_inputs,encoder_states)

decoder_state_input_h=Input(shape=(latent_dim,))
decoder_state_input_c=Input(shape=(latent_dim,))
decoder_states_inputs=[decoder_state_input_h,decoder_state_input_c]

decoder_outputs,state_h,state_c=decoder_lstm(decoder_inputs,initial_state=decoder_states_inputs)
decoder_states=[state_h,state_c]
decoder_outputs=decoder_dense(decoder_outputs)

decoder_model=Model([decoder_inputs]+decoder_states_inputs,[decoder_outputs]+decoder_states)

In [31]:
reverse_input_char_index=dict((i,char) for char ,i in input_token_index.items())
reverse_target_char_index=dict((i,char) for char, i in target_token_index.items())

In [35]:
def decode_sequence(input_seq):
  state_value=encoder_model.predict(input_seq)

  target_seq=np.zeros((1,1,num_decoder_tokens))

  target_seq[0,0,target_token_index['\t']]=1.
  stop_condition=False
  decoded_sentence=' '
  while not stop_condition:
    output_tokens,h,c=decoder_model.predict([target_seq]+state_value)

    sampled_token_index=np.argmax(output_tokens[0,-1,:])
    sampled_char=reverse_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char

    if (sampled_char =='\n' or len(decoded_sentence)>max_decoder_seq_length):
      stop_condition=True

    target_seq=np.zeros((1,1,num_decoder_tokens))
    target_seq[0,0,sampled_token_index]=1.

    state_value=[h,c]
  return decoded_sentence

In [36]:
for seq_index in range(100):
  input_seq=encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence=decode_sequence(input_seq)
  print('-')
  print('Input Sentence ',input_texts[seq_index])
  print('decoded sentence ',decoded_sentence)

-
Input Sentence  Go.
decoded sentence   Bouge !

-
Input Sentence  Go.
decoded sentence   Bouge !

-
Input Sentence  Go.
decoded sentence   Bouge !

-
Input Sentence  Hi.
decoded sentence   Salut.

-
Input Sentence  Hi.
decoded sentence   Salut.

-
Input Sentence  Run!
decoded sentence   Filez !

-
Input Sentence  Run!
decoded sentence   Filez !

-
Input Sentence  Run!
decoded sentence   Filez !

-
Input Sentence  Run!
decoded sentence   Filez !

-
Input Sentence  Run!
decoded sentence   Filez !

-
Input Sentence  Run!
decoded sentence   Filez !

-
Input Sentence  Run!
decoded sentence   Filez !

-
Input Sentence  Run!
decoded sentence   Filez !

-
Input Sentence  Run.
decoded sentence   File !

-
Input Sentence  Run.
decoded sentence   File !

-
Input Sentence  Run.
decoded sentence   File !

-
Input Sentence  Run.
decoded sentence   File !

-
Input Sentence  Run.
decoded sentence   File !

-
Input Sentence  Run.
decoded sentence   File !

-
Input Sentence  Run.
decoded sentence   Fi